In [1]:
import gym, time, math, copy, ray
import numpy as np
import matplotlib.pyplot as plt
ray.shutdown()
ray.init()
time.sleep(2)

2019-07-04 00:03:20,354	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-04_00-03-20_354598_21961/logs.
2019-07-04 00:03:20,460	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:46711 to respond...
2019-07-04 00:03:20,580	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:55668 to respond...
2019-07-04 00:03:20,585	INFO services.py:806 -- Starting Redis shard with 1.64 GB max memory.
2019-07-04 00:03:20,606	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-04_00-03-20_354598_21961/logs.
2019-07-04 00:03:20,609	INFO services.py:1442 -- Starting the Plasma object store with 2.47 GB memory using /dev/shm.


In [2]:
from gym.wrappers import Monitor

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class gameAI(nn.Module):
        def __init__(self):
            super().__init__()
            self.fc = nn.Sequential(
                        nn.Linear(game_observation ,4, bias=True),
                        nn.ReLU(),
                        nn.Linear(4, game_actions, bias=True),
                        nn.Softmax(dim=1)
                        )

                
        def forward(self, inputs):
            x = self.fc(inputs)
            return x

In [5]:
def init_weights(m):
    
        # nn.Conv2d weights are of shape [16, 1, 3, 3] i.e. # number of filters, 1, stride, stride
        # nn.Conv2d bias is of shape [16] i.e. # number of filters
        
        # nn.Linear weights are of shape [32, 24336] i.e. # number of input features, number of output features
        # nn.Linear bias is of shape [32] i.e. # number of output features
        
        if ((type(m) == nn.Linear) | (type(m) == nn.Conv2d)):
            torch.nn.init.xavier_uniform(m.weight)
            m.bias.data.fill_(0.00)
                

In [6]:
def return_random_agents(num_agents):
    
    agents = []
    for _ in range(num_agents):
        
        agent = gameAI()
        
        for param in agent.parameters():
            param.requires_grad = False
        init_weights(agent)
        
        agents.append(agent)
    return agents
# agents = return_random_agents(250)

In [7]:
@ray.remote
def run1(agents, env):
    reward_agents = []
    for agent in agents:
        agent.eval() # I think to set it to evaluation mode and not to training mode
        observation = env.reset()
        r=0
        for _ in range(250):
            
            inp = torch.tensor(observation).type('torch.FloatTensor').view(1,-1) 
            # tensor([[-0.0033, -0.0465, -0.0114, -0.0097]])
            
            output_probabilities = agent(inp).detach().numpy()[0]
            # [0.4562746 0.5437254]
            
            action = np.random.choice(range(game_actions), 1, p=output_probabilities).item() # can only convert an array of size 1 to a Python scalar
            # 0 or 1
            
            observation, reward, done, info = env.step(action)
            r=r+reward
            if(done):
                break
        reward_agents.append(r)        
        #reward_agents.append(s)
    return reward_agents

# run1([agents[0]], gym.make(game))

In [8]:
# %%time
# this function divides agents into {no_of_cores sets of agents} 
def run_agents(agents):
    envS = []
    for i in range(no_of_cores):
        envS.append(gym.make(game))
    
    agents = np.array(agents)
    agents = agents.reshape(no_of_cores, -1)

    result_ids = []
    for i in range(no_of_cores):
        result_ids.append(run1.remote(agents[i], envS[i]))

    results = ray.get(result_ids)
    results = np.array(results, dtype=int)
    return results.reshape(agents.shape[0]*agents.shape[1], -1)

In [9]:
def run_agents_n_times(agents, runs):
    avg_score = np.zeros((len(agents),1))
    for i in range(runs):
        avg_score += run_agents(agents)
    avg_score/=runs
    avg_score = avg_score.reshape(len(agents))
    return avg_score

In [10]:
def mutate(agent):

    child_agent = copy.deepcopy(agent)
    
    for param in child_agent.parameters():
    
        if(len(param.shape)==4): #weights of Conv2D
            for i0 in range(param.shape[0]):
                for i1 in range(param.shape[1]):
                    for i2 in range(param.shape[2]):
                        for i3 in range(param.shape[3]):
                            param[i0][i1][i2][i3] = mutate_each_param(param[i0][i1][i2][i3])
                            
        elif(len(param.shape)==2): #weights of linear layer
            for i0 in range(param.shape[0]):
                for i1 in range(param.shape[1]):                    
                    param[i0][i1] = mutate_each_param(param[i0][i1])

        elif(len(param.shape)==1): #biases of linear layer or conv layer
            for i0 in range(param.shape[0]):
                param[i0] = mutate_each_param(param[i0])

    return child_agent

def mutate_each_param(param):
    if(np.random.rand()<mutation_rate):
        param += np.random.randn()
    return param

In [11]:
def add_elite(agents, sorted_parent_indexes, elite_index=None):
    only_consider_top_n = no_of_cores*2
    print('Only considering top ',only_consider_top_n, ' for elite selection.')
    candidate_elite_index = sorted_parent_indexes[:only_consider_top_n]
    if(elite_index is not None):
        candidate_elite_index = np.append([elite_index], candidate_elite_index)
    candidate_elite_index = candidate_elite_index[:only_consider_top_n]
    # [74, 105, 52, 278, 892, .., 645] 
    
    candidate_elite_agents = []
    for i in candidate_elite_index:
        candidate_elite_agents.append(agents[i])
    candidate_elite_agents = np.array(candidate_elite_agents)
    times = 5
    rewards = run_agents_n_times(candidate_elite_agents, times)
    # [score, score, score, .., score]
    print('Running each elite candidate ', times, ' times.')
    
    top_score = None
    top_elite_index = None
    
    for i in range(len(rewards)):
        score = rewards[i]
        print("Score for elite i ", candidate_elite_index[i], " is ", score)
        
        if(top_score is None):
            top_score = score
            top_elite_index = candidate_elite_index[i]
        elif(score >= top_score):
            top_score = score
            top_elite_index = candidate_elite_index[i]
            
    print("Elite selected with index ",top_elite_index, " and score", top_score)
    
    child_agent = copy.deepcopy(agents[top_elite_index])
    return child_agent
    

In [12]:
def return_children(agents, sorted_parent_indexes, elite_index):
    
    children_agents = []
    
    #first take selected parents from sorted_parent_indexes and generate N-1 children
    for i in range(len(agents)-1):
        
        selected_agent_index = sorted_parent_indexes[np.random.randint(len(sorted_parent_indexes))]
        children_agents.append(mutate(agents[selected_agent_index])) #add mutate here

    #now add one elite
    elite_child = add_elite(agents, sorted_parent_indexes, elite_index)
    children_agents.append(elite_child)
    elite_index=len(children_agents)-1 #it is the last one
    
    return children_agents, elite_index

# return_children(agents, sorted_parent_indexes, elite_index)
# print("ADD MUTATE !!!!!!!!!!!!!!!!!!!!!!!!")

In [13]:
game = 'CartPole-v0'
game_observation = 4
game_actions = 2

#disable gradients as we will not use them
torch.set_grad_enabled(False)

no_of_cores = 8
# initialize N number of agents
num_agents = 400
agents = return_random_agents(num_agents)

# How many top agents to consider as parents
top_limit = 20

mutation_rate = 0.05
#     mutation_power = 0.02 #hyper-parameter, set from https://arxiv.org/pdf/1712.06567.pdf  

# run evolution until X generations
generations = 15

elite_index = None

for generation in range(generations):

    # return rewards of agents
    rewards = run_agents_n_times(agents, 3) #return average of 3 runs

    # sort by rewards
    sorted_parent_indexes = np.argsort(rewards)[::-1][:top_limit] #reverses and gives top values (argsort sorts by ascending by default) https://stackoverflow.com/questions/16486252/is-it-possible-to-use-argsort-in-descending-order
    # [ 92 785 321 682 342  27 946 464  41  21 867 774 893 431 628 399 997 708 820 739]
    
    print("")
    print("")
    
    top_rewards = []
    for best_parent in sorted_parent_indexes:
        top_rewards.append(rewards[best_parent])
    
    print("Generation ", generation, " | Mean rewards: ", int(np.mean(rewards)), " | Mean of top 5: ",int(np.mean(top_rewards[:5])), " | Mean of top ",top_limit," : ", int(np.mean(top_rewards[:top_limit])))
    print("Top ",top_limit," scores", sorted_parent_indexes)
    print("Rewards for top: ",np.array(top_rewards).astype(int))
    
    # setup an empty list for containing children agents
    children_agents, elite_index = return_children(agents, sorted_parent_indexes, elite_index)

    # kill all agents, and replace them with their children
    agents = children_agents

2019-07-04 00:03:24,729	WARNING worker.py:342 -- WARNING: Falling back to serializing objects of type <class 'torch.nn.backends.thnn.THNNFunctionBackend'> by using pickle. This may be inefficient.
2019-07-04 00:03:24,862	WARNING worker.py:342 -- WARNING: Falling back to serializing objects of type <class 'mtrand.RandomState'> by using pickle. This may be inefficient.
2019-07-04 00:03:24,865	WARNING worker.py:342 -- WARNING: Falling back to serializing objects of type <class 'numpy.dtype'> by using pickle. This may be inefficient.




Generation  0  | Mean rewards:  19  | Mean of top 5:  46  | Mean of top  20  :  37
Top  20  scores [262 195  35  28 293  10  95 273 352 181 250  96 227  81  31 356 278 326
 305 196]
Rewards for top:  [56 47 45 41 41 40 39 39 35 35 33 33 33 33 33 33 32 32 32 31]
Only considering top  16  for elite selection.
Running each elite candidate  5  times.
Score for elite i  262  is  32.6
Score for elite i  195  is  21.2
Score for elite i  35  is  21.6
Score for elite i  28  is  17.4
Score for elite i  293  is  16.4
Score for elite i  10  is  29.2
Score for elite i  95  is  26.0
Score for elite i  273  is  31.0
Score for elite i  352  is  17.2
Score for elite i  181  is  23.8
Score for elite i  250  is  26.8
Score for elite i  96  is  18.0
Score for elite i  227  is  24.2
Score for elite i  81  is  19.8
Score for elite i  31  is  21.4
Score for elite i  356  is  17.2
Elite selected with index  262  and score 32.6


Generation  1  | Mean rewards:  22  | Mean of top 5:  52  | Mean of top  20  : 



Generation  9  | Mean rewards:  88  | Mean of top 5:  176  | Mean of top  20  :  163
Top  20  scores [ 35 125  45  82 373 266 154 142 273   3 366 230 160 374 217 157 228 126
  46 175]
Rewards for top:  [182 180 176 173 171 169 168 167 166 164 162 157 157 156 156 156 154 154
 153 153]
Only considering top  16  for elite selection.
Running each elite candidate  5  times.
Score for elite i  399  is  108.0
Score for elite i  35  is  124.0
Score for elite i  125  is  156.6
Score for elite i  45  is  173.6
Score for elite i  82  is  136.8
Score for elite i  373  is  126.6
Score for elite i  266  is  136.0
Score for elite i  154  is  146.6
Score for elite i  142  is  151.0
Score for elite i  273  is  129.4
Score for elite i  3  is  143.0
Score for elite i  366  is  83.8
Score for elite i  230  is  106.6
Score for elite i  160  is  107.4
Score for elite i  374  is  88.6
Score for elite i  217  is  88.8
Elite selected with index  45  and score 173.6


Generation  10  | Mean rewards:  104  | M

In [14]:
import time
def play_agent(agent):
    try: #try and exception block because, render hangs if an erorr occurs, we must do env.close to continue working    
        env_record = gym.make(game)
        
#         env_record = Monitor(env, './video', force=True)
        observation = env_record.reset()
        last_observation = observation
        r=0
        for _ in range(1000):
            env_record.render()
            inp = torch.tensor(observation).type('torch.FloatTensor').view(1,-1)
            output_probabilities = agent(inp).detach().numpy()[0]
            action = np.random.choice(range(game_actions), 1, p=output_probabilities).item()
            new_observation, reward, done, info = env_record.step(action)
            r=r+reward
            observation = new_observation
            xpos = observation[0]
#             print(reward)
#             time.sleep(0.1)
            if(xpos>=2.4 or xpos<=-2.4):
                env_record.reset()

        env_record.close()
        print("Rewards: ",r)

    except Exception as e:
        env_record.close()
        print(e.__doc__)
        print(e.message)        

In [15]:
play_agent(agents[elite_index])

/home/nishantkr18/.local/lib/python3.5/site-packages/gym/logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Rewards:  976.0
